### Задание

Реализуйте алгоритм гребневой регрессии.

Требования к коду:
* Код должен быть хорошо структурирован
* Код должен быть эффективен
* Имплементация должна быть максимально векторизованной (с использованием библиотеки numpy) и не использовать циклы

Необходимо реализовать класс RidgeAnalyticalRegressor, с реализацией прототипа, представленного ниже.

В качестве входного файла необходимо использовать файл "regr_data_XXX.npy", полученный от бота командой /get seminar04

В качестве решения необходимо отправить боту, указав seminar04 в поле caption, следующие файлы:
* ridge_regression.ipynb - этот ноутбук
* ridge_results.npy - файл с результатами тестов, который можно будет сгенерировать с помощью этого ноутбука

Для проверки решения после отправки необходимо отправить боту следующую команду:
/check seminar04

В случае возникновения вопросов по интерфейсу смотрите детали реализации класса sklearn.linear_model.Ridge
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html#sklearn.linear_model.Ridge

In [52]:
import numpy as np

In [53]:
class RidgeAnalyticalRegressor(object):
    '''Класс для предсказания действительно-значного выхода по входу - вектору из R^n. 
    Используется линейная регрессия, то есть если вход X \in R^n, вектор весов W \in R^{n+1},
    то значение регрессии - это [X' 1] * W, то есть y = x1*w1 + x2*w2 + xn*wn + wn+1.
    Обучение - подгонка весов W - будет вестись на парах (x, y).
    
    Параметры
    ----------
    l2_coef    : коэффициент l2 регуляризации  
    
    !!! Внимание: вектор весов W должен быть обязательно объявлен как поле класса RidgeAnalyticalRegressor.coef_ !!!
    '''
    def __init__(self, l2_coef=100.0):
        self.coef_ = None
        self.l2_coef = l2_coef
    
    def fit(self, X, y):
        '''Обучение модели.
        
        Параметры
        ----------
        X : двумерный массив признаков размера n_samples x n_features
        y : массив/список правильных значений размера n_samples
        
        Выход
        -------
        Метод обучает веса W
        '''
        # Append ones
        ones = np.ones((X.shape[0], 1))
        X1 = np.concatenate((X, ones), axis=1)

        # Apply least squares method
        X1T = np.transpose(X1)
        aI = self.l2_coef * np.eye(X1.shape[1])
        self.coef_ = np.linalg.inv(X1T @ X1 + aI) @ X1T @ y
        
    def predict(self, X):
        """ Предсказание выходного значения для входных векторов
        
        Параметры
        ----------
        X : двумерный массив признаков размера n_samples x n_features
        
        Выход
        -------
        y : Массив размера n_samples
        """
        # Append ones
        ones = np.ones((X.shape[0], 1))
        X1 = np.concatenate((X, ones), axis=1)
        
        return np.dot(X1, self.coef_)
        
    def score(self, y_gt, y_pred):
        """Возвращает точность регрессии в виде (1 - u/v) - т.е. R^2-score, 
        где u - суммарный квадрат расхождения y_gt с y_pred,
        v - суммарный квадрат расхождения y_gt с матожиданием y_gt
        
        Параметры
        ----------
        y_gt : массив/список правильных значений размера n_samples
        y_pred : массив/список предсказанных значений размера n_samples
        
        Выход
        -------
        accuracy - точность регрессии
        """
        return 1 - (np.linalg.norm(y_gt - y_pred) / np.linalg.norm(y_gt - np.mean(y_gt))) ** 2

In [54]:
def load_file(filename):
    return np.load(filename).item()

In [55]:
input_filename = "input/regr_data_028.npy"
data_dict = load_file(input_filename)

In [57]:
model = RidgeAnalyticalRegressor(l2_coef=1.0)
model.fit(data_dict["X_train"], data_dict["y_train"])
y_predict = model.predict(data_dict["X_test"])
test_score = model.score(data_dict["y_test"], y_predict)

In [58]:
output_filename = "output/results.npy"
result_dict = {
    "input_filename": input_filename,
    "test_score": test_score,
    "coef": model.coef_[:-1],
    "intercept": model.coef_[-1]
}
np.save(output_filename, result_dict)